In [1]:
#################################################
# IMDB ETL - Init
#################################################

import pandas as pd
import numpy  as np
###
# Null date conversion fun
def conv(val):
    if not val:
        return 0    
    try:
        return np.int(val)
    except:        
        return np.int(0)

In [2]:
import pandas as pd
import numpy  as np

#################################################
# Principals
#################################################

P = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t',encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                 #, error_bad_lines=False
                 , dtype={ "tconst": "str" ,
                           "ordering": "str" ,
                           "nconst": "str" ,
                           "category": "str" ,
                           "job": "str", 
                           "characters": "str"         
                         }
                )
P["cat_cln"] = np.where (P['category'].str.contains('actor') | P['category'].str.contains('actress')
                         ,'actor'
                         ,P['category'])
P["cat_cln"] = np.where (P['category'].str.contains('self') | P['category'].str.contains('archive_footage')
                         ,'self',P['cat_cln'])


P.head(5)

,tconst,ordering,nconst,category,job,characters,cat_cln
0,tt0000001,1,nm1588970,self,\N,"[""Self""]",self
1,tt0000001,2,nm0005690,director,\N,\N,director
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N,cinematographer
3,tt0000002,1,nm0721526,director,\N,\N,director
4,tt0000002,2,nm1335271,composer,\N,\N,composer


In [3]:
import pandas as pd
import numpy  as np
#################################################
# Titles crew directors writers
#################################################

Tcrew = pd.read_csv('https://datasets.imdbws.com/title.crew.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t'
                 , encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'  
                 , dtype={ "tconst": "str" ,
                           "directors": "str" ,
                           "writers": "str" ,       
                         }   
                ) 

Tcrew.rename(columns = {  "directors": "nconst"}
                      , inplace = True)
Tcrew.head(5)

,tconst,nconst,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N


In [4]:
#Tcrew[Tcrew['tconst'] == "tt0000335"]['nconst'].str.split(',').str[0]
#Tcrew[Tcrew['tconst'] == "tt0000335"]['nconst'].str.split(',').str[1]
#Tcrew[Tcrew['tconst'] == "tt1745960"]['nconst'].str.split(',').str[0]
#Tcrew[Tcrew['tconst'] == "tt1745960"]['nconst'].str.split(',').str[1] 
Tcrewi= pd.DataFrame(columns = ['tconst', 'ordering', 'nconst', 'category', 'job', 'characters', 'cat_cln'])
Tcrew9= pd.DataFrame(columns = ['tconst', 'ordering', 'nconst', 'category', 'job', 'characters', 'cat_cln'])
for i in range(0, 4):

    print(i)
    Tcrewi['tconst']   = Tcrew['tconst'] 
    Tcrewi['ordering'] =i
    Tcrewi['nconst']   = Tcrew['nconst'].str.split(',').str[i]
    Tcrewi['category'] ='director'
    Tcrewi['cat_cln']  ='director'
    Tcrewi             =Tcrewi.dropna(subset=['nconst'])
    print (Tcrewi.shape)
    Tcrew9 =pd.concat([Tcrew9, Tcrewi], sort=False)
    print (Tcrew9.shape)
    
#Tcrew9=Tcrew9.dropna(subset=['nconst'])

0
(9774873, 7)
(9774873, 7)
1
(1081838, 7)
(10856711, 7)
2
(340664, 7)
(11197375, 7)
3
(152994, 7)
(11350369, 7)


In [5]:
# add crew to principals
#category == 'director'
#var_list = ('tconst' , 'ordering', 'nconst', 'category', 'job' , 'characters' , 'cat_cln') 


#P1 = P[P['category']!= "director"] 
#P1=pd.concat([P1, Tcrew9], sort=False)
#P1.head(3)

P = P[P['category']!= "director"] 
P = pd.concat([P, Tcrew9], sort=False)
P.head(3)

,tconst,ordering,nconst,category,job,characters,cat_cln
0,tt0000001,1,nm1588970,self,\N,"[""Self""]",self
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N,cinematographer
4,tt0000002,2,nm1335271,composer,\N,\N,composer


In [6]:
Tcrew[Tcrew['tconst'] == "tt1024648"].head(15)

,tconst,nconst,writers
1104615,tt1024648,nm0000255,"nm0006516,nm5410196,nm2646452"


In [7]:
print (P.shape)
P = P.drop_duplicates(subset= ['tconst','nconst'], keep="last")
print (P.shape)
P = P.drop(columns=['job', 'characters'])
print (P.shape)
#P1[P1['tconst'] == "tt0000335"].head(15)
P[P['tconst'] == "tt1024648"].head(15)
#Tcrew9[Tcrew9['tconst'] == ""].head(15)
#P1.head(5)

#Tdir[Tdir['tconst'] == "tt1745960"].head(15)
#Tcrew9.head(7)

(60543670, 7)
(60002024, 7)
(60002024, 5)


,tconst,ordering,nconst,category,cat_cln
8250021,tt1024648,10,nm0006035,composer,composer
8250033,tt1024648,2,nm0186505,actor,actor
8250034,tt1024648,3,nm0000422,actor,actor
8250045,tt1024648,4,nm0000273,actor,actor
8250046,tt1024648,5,nm0006516,writer,writer
8250057,tt1024648,6,nm5410196,writer,writer
8250058,tt1024648,7,nm2646452,writer,writer
8250069,tt1024648,8,nm0000123,producer,producer
8250070,tt1024648,9,nm0381416,producer,producer
1104615,tt1024648,0,nm0000255,director,director


In [ ]:
import pandas as pd
import numpy  as np
#################################################
# Titles
#################################################

def conv(val):
    if not val:
        return 0    
    try:
        return np.int(val)
    except:        
        return np.int(0)
    
T = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t'
                 , encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                #, error_bad_lines=False
                 , dtype={ "tconst": "str" ,
                           "titleType": "str" ,
                           "primaryTitle": "str" ,
                           "originalTitle": "str" ,
                           "isAdult": "str", 
                         # "startYear":   "str" , 
                         # "endYear":   "str" ,
                           "runtimeMinutes": "str" ,
                           "genres": "str"             
                         }
                 , converters={'startYear':conv,'endYear':conv}                
                ) 
T["flg_doc"]           = np.where (T['genres'].str.contains('Documentary'),'Documentary'
                                 , np.where (T['genres'].str.contains('Animation'),'Animation','Fiction'))
T["primaryTitle_year"] = T['primaryTitle']  + ' ('+  T['startYear'].map(str)  + ')'
## filter movie 
T = T.loc[T.titleType == 'movie', :]
T = T.loc[T.startYear != 0, :]

In [ ]:
# T.tail(1000)
T[T['tconst'] == "tt1745960"].head(15)

In [ ]:
var_list = ('tconst','primaryTitle' , 'titleType')
#D = newdf.loc[:,('const','title', 'dat_per', 'title_type')]
D = T.loc[:, var_list]
D.head()
pd.get_dummies(D, columns=['titleType']).head(3)


In [ ]:
#################################################
# Names
#################################################

N = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t',encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                 #, error_bad_lines=False
                 , dtype={ "nconst": "str" ,
                           "primaryName": "str" ,
                    #      "birthYear": "str" ,
                    #       "deathYear": "str" ,
                           "primaryProfession": "str", 
                           "knownForTitles": "str"         
                         }
               , converters={'birthYear':conv,'deathYear':conv}   
               )

#N["primaryName_year"] = N['primaryName']  + ' ('+  N['birthYear'].map(str) + ' - ' +  N['deathYear'].map(str)  + ')'

N["primaryName_year"] = N['primaryName']  + np.where(N['birthYear']==0, ' ',  ' ('+  N['birthYear'].map(str) + ' - '  +  np.where (N['deathYear'] == 0, ' ',  N['deathYear'].map(str)) + ')')
N.head(3)


In [ ]:
#################################################
# Ratings
#################################################
R = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t',encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                 #, error_bad_lines=False
                )

R.rename(columns = {  "averageRating": "tconst_imdb_rating"
                      , "numVotes":   "tconst_imdb_num_votes"
                      , "Const": "tconst"}
                      , inplace = True)
R.tail(3)

In [ ]:
############################################ 
### import my Ratings 
############################################ 
myR = pd.DataFrame()
myR = pd.read_csv ("data/ratings.csv", encoding='latin-1')
myR.rename(columns = {  "Your Rating": "tconst_your_rating"
                      , "Num Votes":   "tconst_your_num_votes"
                      , "Const": "tconst"}
                      , inplace = True)
myR.tail(3)

In [ ]:
#################################################
# Join
#################################################
from datetime import datetime
             
imdb = pd.merge(    P
                  , T 
                  , how = 'inner', left_on="tconst", right_on="tconst"
                 )
imdb = pd.merge(     N.loc[:, ('nconst','primaryProfession', 'primaryName', 'birthYear', 'deathYear', 'primaryName_year')]
                  , imdb
                  , how = 'inner', left_on="nconst", right_on="nconst"
                 )
imdb = pd.merge(     R.loc[:, ('tconst','tconst_imdb_rating', 'tconst_imdb_num_votes')]
                  , imdb
                  , how = 'inner', left_on="tconst", right_on="tconst"
                 )
imdb = pd.merge(    myR.loc[:, ('tconst','tconst_your_rating', 'Release Date' )]
                  , imdb
                  , how = 'right', left_on="tconst", right_on="tconst"
                 )
imdb['rate_diff']           = imdb['tconst_your_rating'] - imdb['tconst_imdb_rating']

imdb["age_crew"]            = np.where(imdb['birthYear']==0, ' ',  imdb['startYear'] - imdb['birthYear'])
imdb["last_refresh"]        = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(P.shape, imdb.shape)


In [ ]:
datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
#imdb_dir = 
imdb_dir = imdb.loc[(imdb['cat_cln'] == "director") & (imdb['ordering'] == 0), ('tconst','primaryName')]  
imdb_dir.rename(columns = {  "primaryName": "primaryName_Dir0"}
                           , inplace = True)
imdb_dir.head(4)
imdb = pd.merge(    imdb_dir.loc[:, ('tconst','primaryName_Dir0')]
                  , imdb
                  , how = 'right', left_on="tconst", right_on="tconst"
                 )
print(imdb_dir.shape, imdb.shape)

#imdb[imdb['tconst'] == "tt1745960"].head(15)
#iimdb[imdb['tconst'] == "tt0000335"].head(15)

In [ ]:
#################################################
# add row number for filmmakers with more than one release per year
#################################################
imdb['RN'] = imdb.sort_values(['nconst', 'tconst'], ascending=[True, False]) \
             .groupby(['nconst', 'startYear']) \
             .cumcount() + 1
imdb.tail(10)

In [ ]:
#################################################
#remove rows not needed 
#################################################

X = imdb[imdb['category']!= "production_designer"] 
X = X[X['category']!= "archive_sound"] 
#X = X[X['category']!= "archive_footage"]
print(X.shape, imdb.shape)

#X.sort_values(by=['tconst', 'averageRating'], ascending=False)
#################################################
# export to file
#################################################
X.to_csv('data/imdb.csv'  ,encoding='utf-8')

In [ ]:
X[X['tconst'] == "tt13833688"].head(15)

In [ ]:
#################################################
# Distinct NCONST aggregation
#################################################
import numpy as np
N_avg = X.pivot_table(  index   = ( 'nconst', 'primaryName_year', 'cat_cln')
                   , aggfunc = [ np.mean]                   
                   , values = ["tconst_imdb_rating", "tconst_your_rating"]
      ##             , fill_value = 0
             )
N_sum = X.pivot_table(  index   = ( 'nconst', 'primaryName_year', 'cat_cln')
                   , aggfunc = [ np.sum]                   
                   , values = [ "tconst_imdb_num_votes"]
                   , fill_value = 0
             )
N_max = X.pivot_table(  index   = ( 'nconst', 'primaryName_year', 'cat_cln')
                   , aggfunc = [ np.max]                   
                   , values = [ "startYear"]
                   , fill_value = 0
             )
N_cnt = X.pivot_table(  index   = ( 'nconst', 'primaryName_year', 'cat_cln')
                   , aggfunc = [ 'count']  
                   , values = ["tconst"]
                   , fill_value = 0
             )

N_unique = pd.merge(   N_sum
                     , N_avg 
                     , how = 'inner', left_on=  [ 'nconst', 'primaryName_year', 'cat_cln'], right_on= [ 'nconst', 'primaryName_year', 'cat_cln']
                 )
N_unique = pd.merge(   N_unique
                     , N_cnt
                     , how = 'inner', left_on= [ 'nconst', 'primaryName_year', 'cat_cln'], right_on= [ 'nconst', 'primaryName_year', 'cat_cln']
                 )
N_unique = pd.merge(   N_unique
                     , N_max
                     , how = 'inner', left_on= [ 'nconst', 'primaryName_year', 'cat_cln'], right_on= [ 'nconst', 'primaryName_year', 'cat_cln']
                 )

#format column names - grouping created by pivo table
N_unique.columns = ['_'.join(str(s).strip() for s in col if s) for col in N_unique.columns]

#reset index
N_unique.reset_index(inplace=True)

N_unique.rename(columns = {  "sum_tconst_imdb_num_votes": "tconst_imdb_num_votes"
                           , "mean_tconst_imdb_rating":   "tconst_imdb_rating"
                           , "mean_tconst_your_rating":   "tconst_your_rating"
                           , "count_tconst":              "cnt_tconst"
                           , "amax_startYear":            "max_startYear"}
                           , inplace = True)

In [ ]:
N_unique.head(3)

In [ ]:
# lst = ['nm0000001', 'nm0000002']
# N_unique =N_unique.query('nconst in @lst')
N_unique.to_csv('data/nconst.csv' , header=0 , encoding='utf-8')

In [ ]:
############################################ 
import pandas as pd 
import os

############################################ 
### import festival inforamaion
############################################ 

var_list = ('Const','Title' , 'Description', 'Directors' , 'Genres', 'Release Date', 'URL')
 
############################################ 
### import festival inforamaion
############################################ 

## https://www.imdb.com/list/ls560308483/    Cannes 2022: Official Selections
## https://www.imdb.com/list/ls560342137     Cannes 75
"""
fst = pd.DataFrame()
fst = pd.read_csv ("data/Cannes 2022 Official Selections.csv", encoding='latin-1')
fst = fst.loc[:, var_list]
fst.rename(columns = {    "Const": "tconst"}
                      , inplace = True)

fst['festival_name'] = 'Cannes'
fst['festival_year'] = '2022'
fst['festival_url']  = 'https://www.festival-cannes.com'
"""
fst  = pd.DataFrame()
for i in range(2021,2023+1):
    
    if os.path.exists("data/Cannes Film Festival " + str(i) + ".csv"):
 
        dfx = pd.read_csv (("data/Cannes Film Festival " + str(i) + ".csv"), encoding='latin-1')
        dfx = dfx.loc[:, var_list ]
        dfx['festival_name'] = 'Cannes'
        dfx['festival_year'] = i
        dfx['festival_url']  = 'https://www.festival-cannes.com'

        fst           = fst.append(dfx)                  #add all rouws
    
fst.rename(columns = {"Const": "tconst"}
                      , inplace = True
          )

############################################ 
### import festival inforamaion
############################################ 

## https://www.imdb.com/list/ls565715845   Yesh 2022
fst1 = pd.DataFrame()
fst1 = pd.read_csv ("data/Yesh2022.csv", encoding='latin-1')
fst1 = fst1.loc[:, var_list]
fst1.rename(columns = {    "Const": "tconst"} 
                      , inplace = True)
fst1['festival_name'] = 'Yesh!'
fst1['festival_year'] = '2022'
fst1['festival_url']  = 'https://www.yesh.ch/'


############################################ 
### import ZFF lists 2011 - 2021  
############################################ 
fst2 = pd.DataFrame()
for i in range(2021,2022+1):
    
    if os.path.exists("data/Zurich Film Festival " + str(i) + ".csv"):
 
        dfx = pd.read_csv (("data/Zurich Film Festival " + str(i) + ".csv"), encoding='latin-1')
        dfx = dfx.loc[:, var_list ]
        dfx['festival_name'] = 'ZFF'
        dfx['festival_year'] = i
        dfx['festival_url']  = 'https://zff.com/en/programme/'

        fst2          = fst2.append(dfx)                  #add all rouws
    
fst2.rename(columns = {"Const": "tconst"}
                      , inplace = True
          )

############################################ 
### import festival inforamaion - Sundance
## https://www.imdb.com/list/ls512369774/   sundance 2022
############################################ 

fst3 = pd.DataFrame()
for i in range(2020,2023+1):
    
    if os.path.exists("data/Sundance " + str(i) + " Movie Guide.csv"):
 
        dfx = pd.read_csv (("data/Sundance " + str(i) + " Movie Guide.csv"), encoding='latin-1')
        dfx = dfx.loc[:, var_list ]
        dfx['festival_name'] = 'Sundance'
        dfx['festival_year'] = i
        dfx['festival_url']  = 'https://festival.sundance.org/'

        fst3          = fst3.append(dfx)                  #add all rouws
    
fst3.rename(columns = {"Const": "tconst"}
                      , inplace = True
          )

############################################ 
### import festival inforamaion SXSW
############################################ 

## https://www.imdb.com/list/ls534810559/   sxsw 2022

fst4 = pd.DataFrame()
for i in range(2020,2023+1):
    
    if os.path.exists("data/SXSW " + str(i) + " Festival Guide.csv"):
 
        dfx = pd.read_csv (("data/SXSW " + str(i) + " Festival Guide.csv"), encoding='latin-1')
        dfx = dfx.loc[:, var_list ]
        dfx['festival_name'] = 'SXSW'
        dfx['festival_year'] = i
        dfx['festival_url']  = 'https://www.sxsw.com/festivals/film'

        fst4          = fst4.append(dfx)                  #add all rouws
    
fst4.rename(columns = {    "Const": "tconst"} 
                      , inplace = True)

############################################ 
### import festival inforamaion LFF
############################################ 

## https://www.imdb.com/list/ls567162891/   lff
fst5 = pd.DataFrame()
fst5 = pd.read_csv ("data/Locarno Film Festival 2022.csv", encoding='latin-1')
fst5 = fst5.loc[:, var_list]
fst5.rename(columns = {   "Const": "tconst"}
                      , inplace = True)
fst5['festival_name'] = 'LFF'
fst5['festival_year'] = '2022'
fst5['festival_url']  = 'https://www.locarnofestival.ch/LFF/home.html'

############################################ 
### import festival inforamaion TIFF
############################################ 

## https://www.imdb.com/list/ls567227005/  tiff

fst6 = pd.DataFrame()
for i in range(2020,2023+1):
    
    if os.path.exists("data/Toronto International Film Festival " + str(i) + ".csv"):
 
        dfx = pd.read_csv (("data/Toronto International Film Festival " + str(i) + ".csv"), encoding='latin-1')
        dfx = dfx.loc[:, var_list ]
        dfx['festival_name'] = 'TIFF'
        dfx['festival_year'] = i
        dfx['festival_url']  = 'https://www.tiff.net/'

        fst6          = fst6.append(dfx)                  #add all rouws
    
fst6.rename(columns = {    "Const": "tconst"} 
                      , inplace = True)

############################################ 
### import Oscar lists   
### https://www.imdb.com/list/ls080103875/ oscars
############################################ 
fst8 = pd.DataFrame()
for i in range(2021,2024+1):
    
    if os.path.exists("data/" + str(i) + " Oscars - Academy Awards predictions.csv"):
 
        dfx = pd.read_csv (("data/" + str(i) + " Oscars - Academy Awards predictions.csv"), encoding='latin-1')
        dfx = dfx.loc[:, var_list ]
        dfx['festival_name'] = 'Oscars'
        dfx['festival_year'] = i
        dfx['festival_url']  = 'https://www.oscars.org/oscars/ceremonies'

        fst8          = fst8.append(dfx)                  #add all rouws
    
fst8.rename(columns = {"Const": "tconst"}
                      , inplace = True
          )
############################################ 
### import Venice lists   
### https://www.imdb.com/list/ls560492207/ oscars
############################################ 
fst9 = pd.DataFrame()
for i in range(2021,2023+1):
    
    if os.path.exists("data/" + str(i) + " Venice International Film Festival.csv"):
 
        dfx = pd.read_csv (("data/" + str(i) + " Venice International Film Festival.csv"), encoding='latin-1')
        dfx = dfx.loc[:, var_list ]
        dfx['festival_name'] = 'Venice'
        dfx['festival_year'] = i
        dfx['festival_url']  = 'https://www.labiennale.org/en/cinema/'

        fst9          = fst9.append(dfx)                  #add all rouws
    
fst9.rename(columns = {"Const": "tconst"}
                      , inplace = True
          )

############################################ 
### import Berlin lists   
### https://www.imdb.com/list/ls560492207/ oscars
############################################ 
fst10 = pd.DataFrame()
for i in range(2021,2023+1):
    
    if os.path.exists("data/" + str(i) + " Berlin International Film Festival.csv"):
 
        dfx = pd.read_csv (("data/" + str(i) + " Berlin International Film Festival.csv"), encoding='latin-1')
        dfx = dfx.loc[:, var_list ]
        dfx['festival_name'] = 'Berlin'
        dfx['festival_year'] = i
        dfx['festival_url']  = 'https://www.berlinale.de/en/home.html'

        fst10          = fst10.append(dfx)                  #add all rouws
    
fst10.rename(columns = {"Const": "tconst"}
                      , inplace = True
          )
############################################ 
### import festival inforamaion
############################################ 

## https://www.imdb.com/list/ls567153978/   what's on
fstw = pd.DataFrame()
fstw = pd.read_csv ("data/whatson.csv", encoding='latin-1')
fstw = fstw.loc[:, var_list]
fstw.rename(columns = {   "Const": "tconst"}
                      , inplace = True)
fstw['festival_name'] = 'WhatsOn'
fstw['festival_year'] = '2023'
fstw['festival_url']  = 'https://www.kulturinfo.ch/kino/index.html'

############################################ 
### import The Year in Movies
############################################ 
fst0 = pd.DataFrame()
for i in range(2021,2022+1):
    
    if os.path.exists("data/" + str(i) + " - The year in movies.csv"):
 
        dfx = pd.read_csv (("data/" + str(i) + " - The year in movies.csv"), encoding='latin-1')
        dfx = dfx.loc[:, var_list ]
        dfx['festival_name'] = 'YTD'
        dfx['festival_year'] = i
        dfx['festival_url']  = 'https://www.kulturinfo.ch/kino/index.html'

        fst0          = fst0.append(dfx)                  #add all rouws
    
fst0.rename(columns = {"Const": "tconst"}
                      , inplace = True
          )

############################################ 
### Union
############################################ 
union = pd.concat([fst, fst1, fst2, fst3, fst4, fst5, fst6, fst8, fst9, fst10,  fstw, fst0])
union.tail(3)


In [ ]:
fst3.head(13)

In [ ]:
          
Xfst = pd.merge(    X
                  , union 
                  , how = 'inner', left_on="tconst", right_on="tconst"
       #         , how = 'right', left_on="tconst", right_on="tconst"
                 )
Xfst.to_csv('data/Xfst.csv' , header=1 , encoding='utf-8')

In [ ]:
#Xfst[Xfst[tconst =='tt10293406']].head(3)
Xfst[Xfst['tconst'] == "tt10293406"] 

In [ ]:

union[union['festival_name'] == "ZFF"].head(15)  

In [ ]:

# duplicate for rating dimension

import numpy as np
# Xfst2 = pd.DataFrame({'Xfst':['tconst','nconst'],})
# Xfst2 = pd.DataFrame({'Xfst':['tconst','nconst'],})
Xfst2 = Xfst.copy()   # panda refernces tables only, so explicit copy required
  
Xfst2['rating_category']           = 'IMDb'
Xfst2['tconst_num_votes']          = np.where(Xfst2['tconst_imdb_num_votes'].isnull(), 0, Xfst2['tconst_imdb_num_votes'])   
Xfst2['tconst_rating']             = np.where(Xfst2['tconst_imdb_rating'].isnull(), np.nan, Xfst2['tconst_imdb_rating'] )   
Xfst2.head(3)

Xfst3 = Xfst.copy()
  
Xfst3['rating_category']           = 'Profile_1'
#Xfst3['tconst_num_votes']          = 1   
#Xfst3['tconst_rating']             = np.where(Xfst3['tconst_your_rating'].isnull(), 0, Xfst3['tconst_your_rating'] )
Xfst3['tconst_num_votes']       = np.where(Xfst3['tconst_your_rating'].isnull(), 0, 1)
Xfst3['tconst_rating']          = np.where(Xfst3['tconst_your_rating'].isnull(), np.nan, Xfst3['tconst_your_rating'] )
Xfst3.head(3)
 

 
rating_duplicate = pd.concat([Xfst2, Xfst3], ignore_index=True) 
rating_duplicate.to_csv('data/rating_duplicate.csv' , header=1 , encoding='utf-8')

In [ ]:
rating_duplicate.tail(4)

In [ ]:
N_unique.head(3)

In [ ]:
# duplicate for rating dimension



N2 = N_unique.copy()   # panda refernces tables only, so explicit copy required
  
N2['rating_category']           = 'IMDb'
N2['tconst_num_votes']          = np.where(N2['tconst_imdb_num_votes'].isnull(), 0, N2['tconst_imdb_num_votes'] )  
N2['tconst_rating']             = np.where(N2['tconst_imdb_rating'].isnull(), np.nan, N2['tconst_imdb_rating'] )   
N2.head(3)

N3 = N_unique.copy()
  
N3['rating_category']           = 'Profile_1'
N3['tconst_num_votes']          = np.where(N3['tconst_your_rating'].isnull(), 0, 1 )
N3['tconst_rating']             = np.where(N3['tconst_your_rating'].isnull(), np.nan, N3['tconst_your_rating'] )
N3.head(3)
 


nconst_duplicate = pd.concat([N2, N3], ignore_index=True) 
nconst_duplicate.to_csv('data/nconst_duplicate.csv' , header=1 , encoding='utf-8')


In [ ]:
print(fst.shape, Xfst.shape)
nconst_duplicate.head(3)

In [ ]:
# Xfst.tail(3)
Xfst[Xfst['tconst'] == "tt13833688"].head(15)